In [2]:
import pyclesperanto_prototype as cle

from skimage.io import imread, imsave

# initialize GPU
device = cle.select_device("GTX")
print("Used GPU: ", device)

# load data
image = imread('https://imagej.nih.gov/ij/images/blobs.gif')

# process the image
inverted = cle.subtract_image_from_scalar(image, scalar=255)
blurred = cle.gaussian_blur(inverted, sigma_x=1, sigma_y=1)
binary = cle.threshold_otsu(blurred)
labeled = cle.connected_components_labeling_box(binary)

# The maxmium intensity in a label image corresponds to the number of objects
num_labels = cle.maximum_of_all_pixels(labeled)

# print out result
print("Num objects in the image: " + str(num_labels))

# save image to disc
imsave("result.tif", labeled)

Used GPU:  <Apple M1 on Platform: Apple (2 refs)>
Num objects in the image: 62.0


/var/folders/vg/pkq1xs_x1b559nk61kjgcgkh0000gn/T/ipykernel_23749/2795466400.py:25: UserWarning: result.tif is a low contrast image
  imsave("result.tif", labeled)


In [22]:
import pyopencl as cl
import napari_segment_blobs_and_things_with_membranes as nsbatwm  # version 0.3.3
from napari_simpleitk_image_processing import label_statistics
import napari_simpleitk_image_processing

In [9]:
zoom=1
lmsw_3d = nsbatwm.local_minima_seeded_watershed(image, 25.0*zoom, 5.0*zoom)

In [15]:
# exclude labels on the edge of the stack
exclude_labels_on_edges = cle.exclude_labels_on_edges(lmsw_3d)

In [20]:
cle.select_device('CPU')

<Apple M1 on Platform: Apple (2 refs)>

In [23]:
dir(napari_simpleitk_image_processing)

['__builtins__',
 '__cached__',
 '__common_alias__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_function',
 '_simpleitk_image_processing',
 'adaptive_histogram_equalization',
 'bilateral_filter',
 'binary_fill_holes',
 'binominal_blur_filter',
 'black_top_hat',
 'canny_edge_detection',
 'connected_component_labeling',
 'curvature_flow_denoise',
 'elongation_map',
 'feret_diameter_map',
 'gaussian_blur',
 'gradient_magnitude',
 'h_maxima',
 'h_minima',
 'invert_intensity',
 'label_contour',
 'label_statistics',
 'laplacian_filter',
 'laplacian_of_gaussian_filter',
 'median_filter',
 'morphological_gradient',
 'morphological_watershed',
 'napari_experimental_provide_function',
 'otsu_multiple_thresholds',
 'pixel_count_map',
 'plugin_function',
 'regional_maxima',
 'regional_minima',
 'relabel_component',
 'rescale_intensity',
 'richardson_lucy_deconvolution',
 'roundness_map',
 'scalar_image_k_means_clustering',
 'si

In [17]:
statistics = label_statistics(image, 
                              exclude_labels_on_edges, 
                              intensity=True, 
                              size=False, 
                              shape=False, 
                              perimeter=False, 
                              position=False,
                              moments=False
                             )

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'Apple M1' on 'Apple' at 0x60000239cdc0>:

Compiler encountered an internal error
(options: -I /Users/jwhite/miniforge3/envs/tf-napari/lib/python3.10/site-packages/pyopencl/cl)
(source saved as /var/folders/vg/pkq1xs_x1b559nk61kjgcgkh0000gn/T/tmpcsh6w4cq.cl)

In [12]:
! more /var/folders/vg/pkq1xs_x1b559nk61kjgcgkh0000gn/T/tmp1cf74wqw.cl

#define MAX_ARRAY_SIZE 1000
#define GET_IMAGE_WIDTH(image_key) image_size_ ## image_key ## _width
#define GET_IMAGE_HEIGHT(image_key) image_size_ ## image_key ## _height
#define GET_IMAGE_DEPTH(image_key) image_size_ ## image_key ## _depth

#define CONVERT_dst_PIXEL_TYPE clij_convert_long_sat
#define IMAGE_dst_PIXEL_TYPE long
#define POS_dst_TYPE int2
#define POS_dst_INSTANCE(pos0,pos1,pos2,pos3) (int2)(pos0, pos1)

#define IMAGE_dst_TYPE int image_size_dst_width, int image_size_dst_height, int image_size_dst_depth,  __global long*
#define READ_dst_IMAGE(a,b,c) read_buffer2dl(GET_IMAGE_WIDTH(a),GET_IMAGE_HEIGHT(a),GET_IMAGE_DEPTH(a),a,b,c)
#define WRITE_dst_IMAGE(a,b,c) write_buffer2dl(GET_IMAGE_WIDTH(a),GET_IMAGE_HEIGHT(a),GET_IMAGE_DEPTH(a),a,b,c)


#define CONVERT_src_PIXEL_TYPE clij_convert_uint_sat
#define IMAGE_src_PIXEL_TYPE uint
#define POS_src_TYPE int2
#define POS_src_INSTANCE(pos0,pos1,pos2,pos3) (int2)(pos0, pos1)

>olders/vg/pkq1xs_x1b559nk61kjgcgkh0000gn/T/tmp1cf74wqw.cl